<a href="https://colab.research.google.com/github/chandunulu/Anime-verse/blob/main/final_food_delivery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd


In [2]:
ord=pd.read_csv('orders.csv')
ord.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [3]:
user=pd.read_json('users.json')
user.head()

,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [15]:
import sqlite3
import pandas as pd
con=sqlite3.connect("restaurants.db")
cursor=con.cursor()
with open("restaurants.sql","r") as file:
  sql_script=file.read()
cursor.execute("DROP TABLE IF EXISTS restaurants;")
cursor.executescript(sql_script)
con.commit()

restaurants=pd.read_sql_query("SELECT * FROM restaurants",con)
restaurants.head()

,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [19]:
df=pd.merge(ord,user,on='user_id', how='left')
df=pd.merge(df,restaurants,on='restaurant_id',how='left')
df.head(10)
df.to_csv('final_food_delivery_dataset.csv',index=False)



In [20]:
df.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [59]:
g=df[df['membership']=='Gold']
gf=g.groupby("city")["total_amount"].sum()
gf.idxmax()

'Chennai'

In [60]:
avg=df.groupby('cuisine')['total_amount'].mean()
avg.idxmax()

'Mexican'

In [27]:
u=df.groupby('user_id')['total_amount'].sum()
hu=u[u>1000]
hu.count()

np.int64(2544)

In [29]:
res=df.groupby('cuisine')['restaurant_id'].nunique()
re=df.groupby('cuisine')['total_amount'].sum()
r=pd.DataFrame({'Unique_Restaurants': res, 'Total_Revenue': re})
r

,Unique_Restaurants,Total_Revenue
cuisine,,
Chinese,120,1930504.65
Indian,126,1971412.58
Italian,126,2024203.80
Mexican,128,2085503.09


In [61]:
av=df[df['membership']=='Gold']
g=av.groupby("city")["total_amount"].mean()
gf.idxmax()

'Chennai'

In [34]:
total_orders=len(df)
gold_orders=df[df['membership']=='Gold']
totalgold_orders=len(gold_orders)
per=(totalgold_orders/total_orders)*100
per

49.87

In [48]:
subset = df[df['restaurant_name_x'].isin([
    'Grand Cafe Punjabi',
    'Grand Restaurant South Indian',
    'Ruchi Mess Multicuisine',
    'Ruchi Foods Chinese'
])]

rs=subset.groupby('restaurant_name_x').agg(
    average_order_value=('total_amount', 'mean'),
    total_orders=('order_id', 'count')
).reset_index()
fs = rs[rs['total_orders'] < 20]
t=fs.loc[fs['average_order_value'].idxmax()]
print(t)



restaurant_name_x      Ruchi Foods Chinese
average_order_value             686.603158
total_orders                            19
Name: 2, dtype: object


In [49]:
df.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [64]:
ce=df.groupby(['membership','cuisine'])['total_amount'].sum()
ce =ce.loc[[('Gold', 'Indian'), ('Gold', 'Italian'), ('Regular', 'Indian'), ('Regular', 'Chinese')]]
ce.idxmax()

('Gold', 'Italian')

In [65]:
df['quarter'] = df['order_date'].dt.quarter
revenue=df.groupby('quarter')['total_amount'].sum()
revenue.idxmax()

np.int32(3)

In [68]:
gm=df[df['membership']=='Gold']
len(gm)

4987

In [75]:
ci=df[df['city']=='Hyderabad']
ci['total_amount'].sum()


np.float64(1889366.58)

In [76]:
ui=df.groupby('user_id').nunique()
len(ui)

2883

In [77]:
go=df[df['membership']=='Gold']
go['total_amount'].mean()


np.float64(797.1455564467616)

In [78]:
re=df[df['rating']>=4.5]
len(re)

3374

In [81]:
gol=df[df['membership']=='Gold']
rev=gol.groupby('city')['total_amount'].sum()
ci=rev.idxmax()
fi=gol[gol['city']==ci]['order_id'].count()
fi

np.int64(1337)

In [84]:
len(df)

10000